In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


* This data is combined(across the years and states) and largely clean version of the Historical Daily Ambient Air Quality Data released by the Ministry of Environment and Forests and Central Pollution Control Board of India under the National Data Sharing and Accessibility Policy (NDSAP).
* Detect pollution trends

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot

In [ ]:
df = pd.read_csv("/kaggle/input/india-air-quality-data/data.csv", encoding = "ISO-8859-1")
df.head()

* **stn_code** : Station code. A code given to each station that recorded the data.
* **sampling_date** : The date when the data was recorded.
* **state** : It represents the states whose air quality data is measured.
* **location** : It represents the city whose air quality data is measured.
* **agency** : Name of the agency that measured the data.
* **type** : The type of area where the measurement was made.
* **so2** : The amount of Sulphur Dioxide measured.
* **no2** : The amount of Nitrogen Dioxide measured
* **rspm** : Respirable Suspended Particulate Matter measured.
* **spm** : Suspended Particulate Matter measured.
* **location_monitoring_station** : It indicates the location of the monitoring area.
* **pm2_5** : It represents the value of particulate matter measured.
* **date** : It represents the date of recording (It is cleaner version of ‘sampling_date’ feature)


In [ ]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d') # date parse
df['year'] = df['date'].dt.year # year
df['year'] = df['year'].fillna(df["year"].min())
df['year'] = df['year'].values.astype(int)

In [ ]:
print (df.get_dtype_counts())

# EDA

## Null Values

In [ ]:
def printNullValues(df):
    total = df.isnull().sum().sort_values(ascending = False)
    total = total[df.isnull().sum().sort_values(ascending = False) != 0]
    percent = total / len(df) * 100
    percent = percent[df.isnull().sum().sort_values(ascending = False) != 0]
    concat = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
    print (concat)
    print ( "-------------")

In [ ]:
printNullValues(df)

* agency’s name have nothing to do with how much polluted the state is. 
* stn_code is also unnecessary.
* date and sampling_date are similar
* location_monitoring_station 

## Type

In [ ]:
df["type"].value_counts()

In [ ]:

sns.catplot(x = "type", kind = "count",  data = df, height=5, aspect = 4)

### Analysis by type and pollution means

In [ ]:
grp = df.groupby(["type"]).mean()["so2"].to_frame()
grp.plot.bar(figsize = (20,10))

In [ ]:
grp = df.groupby(["type"]).mean()["no2"].to_frame()
grp.plot.bar(figsize = (20,10))

## SO2

Sulfur dioxide

In [ ]:

df[['so2', 'state']].groupby(['state']).median().sort_values("so2", ascending = False).plot.bar(figsize=(20,10))


In [ ]:
df[['so2','year','state']].groupby(["year"]).median().sort_values(by='year',ascending=False).plot(figsize=(20,10))

## no2 

Nitrogen dioxide

In [ ]:

df[['no2', 'state']].groupby(['state']).median().sort_values("no2", ascending = False).plot.bar(figsize=(20,10))


In [ ]:
df[['no2','year','state']].groupby(["year"]).median().sort_values(by='year',ascending=False).plot(figsize=(20,10))

## SPM

 Suspended Particulate Matter

In [ ]:

df[['spm', 'state']].groupby(['state']).median().sort_values("spm", ascending = False).plot.bar(figsize=(20,10))


In [ ]:
df[['spm','year','state']].groupby(["year"]).median().sort_values(by='year',ascending=False).plot(figsize=(20,10))

## PIVOT tables

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))      
sns.heatmap(df.pivot_table('so2', index='state',columns=['year'],aggfunc='median',margins=True),ax = ax,annot=True, linewidths=.5)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))      
sns.heatmap(df.pivot_table('no2', index='state',columns=['year'],aggfunc='median',margins=True),ax = ax,annot=True, linewidths=.5)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))      
sns.heatmap(df.pivot_table('spm', index='state',columns=['year'],aggfunc='median',margins=True),ax = ax,annot=False, linewidths=.5)

## Trends by regions

In [ ]:
temp = df.pivot_table('so2', index='year',columns=['state'],aggfunc='median',margins=True).reset_index()
temp = temp.drop("All", axis = 1)
temp = temp.set_index("year")
temp.plot(figsize=(20,10))

In [ ]:
temp = df.pivot_table('no2', index='year',columns=['state'],aggfunc='median',margins=True).reset_index()
temp = temp.drop("All", axis = 1)
temp = temp.set_index("year")
temp.plot(figsize=(20,10))

In [ ]:
temp = df.pivot_table('spm', index='year',columns=['state'],aggfunc='median',margins=True).reset_index()
temp = temp.drop("All", axis = 1)
temp = temp.set_index("year")
temp.plot(figsize=(20,10))

* it would make sense to fill the missing values with the mean between the last two available previous and next value?

# Geoplotting

In [ ]:
india = gpd.read_file('/kaggle/input/maps-of-india/India_SHP/INDIA.shp')
india.info()

In [ ]:
india.plot()

* Match the names of states between the two datasets

In [ ]:
india["ST_NAME"] = india["ST_NAME"].apply(lambda x: x.lower())

india = india.set_index("ST_NAME")

df["state"] = df["state"].apply(lambda x: x.lower())

In [ ]:
df_before_2000 = df[df["year"] < 2000]
df_before_2000 = df_before_2000.groupby("state").mean()

In [ ]:
df_after_2000 = df[df["year"] > 2000]
df_after_2000 = df_after_2000.groupby("state").mean()

In [ ]:
result = pd.concat([df_before_2000, india], axis=1, sort=False)
result = result [result["geometry"] != None]
result = result [result["year"] > 0]
from geopandas import GeoDataFrame
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(result, crs=crs, geometry=result ["geometry"])
gdf['centroid'] = gdf.geometry.centroid
fig,ax = plt.subplots(figsize=(20,10))
gdf.plot(column='so2',ax=ax,alpha=0.4,edgecolor='black',cmap='cool', legend=True)
plt.title("Mean So2 before 2000")
plt.axis('off')

for x, y, label in zip(gdf.centroid.x, gdf.centroid.y, gdf.index):
    ax.annotate(label, xy=(x, y), xytext=(3,3), textcoords="offset points",color='gray')

In [ ]:
result = pd.concat([df_after_2000, india], axis=1, sort=False)
result = result [result["geometry"] != None]
result = result [result["year"] > 0]
from geopandas import GeoDataFrame
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(result, crs=crs, geometry=result ["geometry"])
gdf['centroid'] = gdf.geometry.centroid
fig,ax = plt.subplots(figsize=(20,10))
gdf.plot(column='so2',ax=ax,alpha=0.4,edgecolor='black',cmap='cool', legend=True)
plt.title("Mean So2 after 2000")
plt.axis('off')

for x, y, label in zip(gdf.centroid.x, gdf.centroid.y, gdf.index):
    ax.annotate(label, xy=(x, y), xytext=(3,3), textcoords="offset points",color='gray')

* two maps show trends in So2 pollution mean

# Time Series Analysis

* so2 Sulfur dioxide

In [ ]:
df_so2 = df[["date", "so2"]]
df_so2 = df_so2.set_index("date")
df_so2 = df_so2.dropna()

In [ ]:
df_so2_resample = df_so2.resample(rule = "M").mean().ffill()

In [ ]:
df_so2_resample.plot(figsize = (20,10))

In [ ]:
df_so2_resample["so2"].resample("A").mean().plot.bar(figsize = (20,10))

### ETS Decomposition (Error Trend Seasonality)

* Simple Moving Average

In [ ]:
df_so2_resample.plot(figsize = (20,10))
df_so2_resample.rolling(window = 7).mean()["so2"].plot(figsize = (20,10))

* Esponential weighted moving average EWMA
apply more weight to value more recent

In [ ]:
df_so2_resample["EWMA-7"] = df_so2_resample["so2"].ewm(span=7).mean()

In [ ]:
df_so2_resample.plot(figsize = (20,10))

### ETS



In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df_so2_resample["so2"], model = "multiplicative") 

In [ ]:
fig = result.plot()

## ARIMA and Seasonal ARIMA

#### Autoregressive Integrated Moving Averages

* https://people.duke.edu/~rnau/411arim3.htm ARIMA explained

* Make the time series data stationary
* Plot the Correlation and AutoCorrelation Charts
* Construct the ARIMA Model
* Use the model to make predictions

#### Testing the Stationarity

Basically, we are trying to whether to accept the Null Hypothesis **H0** (that the time series has a unit root, indicating it is non-stationary) or reject **H0** and go with the Alternative Hypothesis (that the time series has no unit root and is stationary).

We end up deciding this based on the p-value return.

* A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis.

* A large p-value (> 0.05) indicates weak evidence against the null hypothesis, so you fail to reject the null hypothesis.



In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(df_so2_resample["so2"])
print('Augmented Dickey-Fuller Test:')
labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

for value,label in zip(result,labels):
    print(label+' : '+str(value) )
    
if result[1] <= 0.05:
    print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
else:
    print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

* the data is seasonal ---> use Seasonal ARIMA

In [ ]:
df_so2_resample["so2_first_diff"] = df_so2_resample["so2"] - df_so2_resample["so2"].shift(7)
# CHECK
result = adfuller(df_so2_resample["so2_first_diff"].dropna() )
print('Augmented Dickey-Fuller Test:')
labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

for value,label in zip(result,labels):
    print(label+' : '+str(value) )
    
if result[1] <= 0.05:
    print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
else:
    print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
df_so2_resample["so2_first_diff"].plot(figsize = (20,10))

In [ ]:
df_so2_resample["so2_second_diff"] = df_so2_resample["so2_first_diff"] - df_so2_resample["so2_first_diff"].shift(7)
df_so2_resample["so2_second_diff"].plot(figsize = (20,10))

## Using the Seasonal ARIMA model

In [ ]:
import statsmodels.api as sm

model = sm.tsa.statespace.SARIMAX(df_so2_resample["so2"],order=(0,1,0), seasonal_order=(1,1,1,48))
results = model.fit()
print(results.summary())
results.resid.plot()

In [ ]:
results.resid.plot(kind='kde')

#### Check with known data 

In [ ]:
df_so2_resample['forecast'] = results.predict(start = 250, end= 400, dynamic= True)  
df_so2_resample[['so2','forecast']].plot(figsize=(20,10))

# Forecast

* so2

In [ ]:
from pandas.tseries.offsets import DateOffset
future_dates = [df_so2_resample.index[-1] + DateOffset(months=x) for x in range(0,24) ]
future_dates_df = pd.DataFrame(index=future_dates[1:],columns=df_so2_resample.columns)
future_df = pd.concat([df_so2_resample,future_dates_df])
future_df['forecast2'] = results.predict(start = 348, end = 540, dynamic= True)  
future_df[['so2', 'forecast2']].plot(figsize=(20, 10)) 